## Importing librairies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
# Verification that GPU is active.
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


Defining global variables

In [3]:
width, height = 256, 256 # size of the images in the dataset
data_dir = "H:/Path/To/Your/Dataset" # path to the dataset for training and validation
data_dir_test = None # path to the dataset for test (not required)
batch_size = 8
augmentation = True # If you want to apply data augmentation to your dataset
epochs = 30 # Number of epoch for training
name_model = 'NameOfYourModel.h5' # The name of the model, it will save the weights of the model as a .h5 file if you want to use the inference later. 

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(height, width),
  batch_size=batch_size)

Found 21897 files belonging to 6 classes.
Using 17518 files for training.


In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(height, width),
  batch_size=batch_size)

Found 21897 files belonging to 6 classes.
Using 4379 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['autre', 'batiment', 'champ', 'eau', 'foret', 'naturel']


Defining useful functions

In [7]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

def time_it(func):
    """
    Usage : Put @time_it before your function to calculate execution time.
    """
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Execution time of {func.__name__}: {end - start:.5f} seconds")
        return result
    return wrapper

Plot of some images to check that everything is going smoothly

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(min(9,batch_size)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds: # Printing the shape of the dataset
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
SEEDS=123

if augmentation:
  data_aug = tf.keras.Sequential([
    tf.keras.layers.RandomContrast(factor=0.1, seed=SEEDS),
    tf.keras.layers.RandomBrightness(factor=0.1, seed=SEEDS)
  ])

  train_ds = train_ds.map(lambda x, y: (data_aug(x, training=True), y))
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Plot augmented data
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(min(9,batch_size)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

## Model definition

Here is a list of available model to use as a backbone : https://keras.io/api/applications/

In [ ]:
num_classes = len(class_names)
# Backbone of the model
base_model = tf.keras.applications.EfficientNetB3(input_shape=(height, width,3), include_top=False ,weights='imagenet')
base_model.trainable = False
base_model.summary()

In [13]:
inputs = keras.Input(shape=(height, width,3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
# Fully connected layer
x = tf.keras.layers.Dense(4096, activation='relu')(x)
# Logistic layer (number of classes)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model1 = tf.keras.Model(inputs, outputs=outputs)

In [15]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
es = EarlyStopping(monitor='val_accuracy', mode='auto', verbose=0,patience=10)
mc = ModelCheckpoint(name_model, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

model1.compile(optimizer=keras.optimizers.Adam(),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# Only train top layers
with tf.device('/device:GPU:0'): # ensure that training is done by GPU just in case
    history = model1.fit(train_ds, validation_data=val_ds,batch_size=batch_size, epochs=epochs-10,verbose=1, callbacks = [es,mc])

In [ ]:
plot_history(history=history)

In [20]:
base_model.trainable = True

# we need to recompile the model for these modifications to take effect
model1.compile(optimizer=keras.optimizers.Adam(1e-4),  # Low learning rate
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [ ]:
history11 = model1.fit(train_ds, validation_data=val_ds,batch_size=batch_size, epochs=epochs,verbose=1, callbacks = [es,mc])

In [ ]:
plot_history(history11)

In [23]:
from keras.models import load_model
saved_model = load_model(name_model)

In [ ]:
if data_dir_test != None:
    test_ds = tf.keras.utils.image_dataset_from_directory(
            data_dir_test,
            seed=123,
            image_size=(height, width),
            batch_size=batch_size)
    evaluation = saved_model.evaluate(test_ds)

### Plot Confusion Matrix

In [ ]:
y_pred = []  # store predicted labels
y_true = []  # store true labels

# iterate over the dataset
for image_batch, label_batch in test_ds:   # use dataset.unbatch() with repeat
   # append true labels
   y_true.append(label_batch)
   # compute predictions
   preds = saved_model.predict(image_batch)
   # append predicted labels
   y_pred.append(np.argmax(preds, axis = - 1))

# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

In [26]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(predicted_labels, correct_labels)

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

# Fill confusion matrix cells with values
thresh = cm.max() / 2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()